# Ejercicio 5
### RSA
**David Cabezas Berrido, Patricia Córdoba Hidalgo, Pilar Navarro Ramírez y Yábir García Benchakhtir**

En primer lugar definimos dos funciones, una para cifrar y otra para descifrar que trabajan con numeros

In [1]:
import math

def RSA_Encrypt(p,q,e,m):    
    # Power_mod utiliza el algoritmo de exponenciacion rapida como se puede ver
    # en el siguiente enlace
    # https://github.com/sagemath/sage/blob/222059565bc2166f29c50a6d85db7992589098c2/src/sage/arith/misc.py#L2201
    return power_mod(m, e, p*q) 

def RSA_Decrypt(p,q,e,m):    
    d = power_mod(e, -1, (p-1)*(q-1))
    return power_mod(m, d, p*q)

Un ejemplo que verifica la correccion de las funciones anteriores

In [2]:
m = RSA_Encrypt(97, 113, 865, 8888)
m, RSA_Decrypt(97, 113, 865, m)

(6269, 8888)

Definimos las funciones _numfy_ y _denumfy_ que utilizaremos para convertir entre caracteres utf-8 y su numero asociado

In [3]:
def numfy(s):
    """
    Función que dado una cadena devuelve un número que representa a 
    dicha cadena. Esta funcion permite codificar mensajes en utf-8
    """
    number = 0
    for e in [ord(c) for c in s]:
        number = (number * 0x110000) + e
    return number

def denumfy(number):
    """
    Obtiene una cadena de texto utf-8 a partir de su representación numérica
    """
    l = []
    while(number != 0):
        l.append(chr(number % 0x110000))
        number = number // 0x110000
    return ''.join(reversed(l))

Función auxiliar que emplearemos para seccionar una cadena de texto en bloques de determinada longitud

In [4]:
def chunkyfy(m, chunk_size ):
    """
    m: str que representa la cadena de texto a partir
    chunk_size: int tamanio de los bloques
    """
    # Aniadimos 0 al principio de la cadena para que todos los bloques
    # tenga la misma longitud
    m = str(m)
    m_encoded = (chunk_size - (len(m)% chunk_size )) % chunk_size * '0' + m
    
    # Construimos los bloques
    f = len(m_encoded)//chunk_size
    return [ int(m_encoded[i*chunk_size:(i+1)*chunk_size]) for i in range(f) ]
    

Ahora creamos funciones _wrapper_ que llaman a las definidas anteriormente pero incorporan la logica para trabajar con cadenas de texto

In [5]:
def RSAe(p,q,e,m):
    n = p*q
    chunk_size = int(math.log(n, 10))
    digits_n = chunk_size + 1
    
    mc = numfy(m)
    chunks = chunkyfy(mc, chunk_size)
    f = "{:0" +  str(digits_n) + "d}"
    encrypted = [f.format(RSA_Encrypt(p,q,e,mi)) for mi in chunks]
    return "".join(encrypted)

In [6]:
def RSAd(p,q,e,m):    
    n = p*q
    chunk_size = int(math.log(n, 10))
    digits_n = chunk_size + 1
    
    chunks = chunkyfy(int(m), digits_n)
    f = "{:0" +  str(chunk_size) + "d}"
    decrypted = [f.format(RSA_Decrypt(p,q,e,mi)) for mi in chunks]
    return denumfy(int("".join(decrypted)))

Comprobamos que efectivamente funcionan las funciones anteriores

In [7]:
m = RSAe(287169250670647, 31472159917268862941, 2312856761, "你好 es lo mismo que привет que significa hola! ❤️")
print(m)

565738486835952364765089729272359016411035886604055013128544468869077319214174904631818845199170632317083769967632736565871287037212704802713826825393871848520074282612536803876572049989286295662762507290771186626595072871033938454376120120287973172269361652278551550069334535147896380659249759712638112171


In [8]:
print(RSAd(287169250670647, 31472159917268862941, 2312856761, m))

你好 es lo mismo que привет que significa hola! ❤️
